In [ ]:
##########
# TO DO #
#########

# 5/21/2024
# Adding artist_genres to the feature dataframe
#  Could add it when adding tracks 
#  or just perform it afterwards
#    like updating in batches as to avoid long run times on API loops

In [11]:
import requests
import time
import ast

import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Loading in Spotify API credentials
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path = 'api_cred/spotify.env')

cid = os.getenv('cid')
secret = os.getenv('secret')

client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [112]:
# Get Access Token

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL,{
    'grant_type': 'client_credentials',
    'client_id': cid,
    'client_secret': secret
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
sp.auth = access_token

print(auth_response_data)

{'access_token': 'BQALjbfb2eRLqNEcyfTIxBccPs15JvinYnFkas_lVU1raev9t46X8WgWbN9ux0PWWwHIw--7Tt9YX1ZCSTAkyPFoNpN6NKZyDxgrVR8KJZH1vc-WZ0k', 'token_type': 'Bearer', 'expires_in': 3600}


# Artists DF

## Get from playlist

In [5]:
def add_artist_name_from_playlist(playlist_url, artists_list):
    try:
        playlist_url = playlist_url.rsplit('/', 1)[-1]
        playlist = sp.playlist(playlist_url)
        for track in playlist['tracks']['items']:
            if track['track']['artists'][0]['name'] not in artists_list:
                artists_list.append(track['track']['artists'][0]['name'])
            else:
                pass
    except spotipy.exceptions.SpotifyException as e:
        print(f'Spotify Exception: {e}')
        print(f'Bad link: {playlist_url}')
    except Exception as e:
        print(f'An error occurred: {e}')

In [85]:
# Only using this to pad out initial list of artists

playlist_get = ['https://open.spotify.com/playlist/37i9dQZF1DX6Q49slBRXI2', 'https://open.spotify.com/playlist/37i9dQZF1DX9lzz0FRAxgl', 
                'https://open.spotify.com/playlist/37i9dQZF1DXdpy4ZQQMZKm', 'https://open.spotify.com/playlist/37i9dQZF1DX4dwwIoKH0j7', 
                'https://open.spotify.com/playlist/37i9dQZF1DX18jTM2l2fJY', 'https://open.spotify.com/playlist/56r5qRUv3jSxADdmBkhcz7', 
                'https://open.spotify.com/playlist/5GhQiRkGuqzpWZSE7OU4Se', 'https://open.spotify.com/playlist/2fmTTbBkXi8pewbUvG3CeZ', 
                'https://open.spotify.com/playlist/37i9dQZF1DWVRSukIED0e9', 'https://open.spotify.com/playlist/37i9dQZF1DXe2bobNYDtW8',
                'https://open.spotify.com/playlist/37i9dQZF1DX3Sp0P28SIer', 'https://open.spotify.com/playlist/37i9dQZF1DXcagnSNtrGuJ', 
                'https://open.spotify.com/playlist/37i9dQZF1DX1vSJnMeoy3V', 'https://open.spotify.com/playlist/37i9dQZF1DWTWdbR13PQYH', 
                'https://open.spotify.com/playlist/37i9dQZF1DXaW8fzPh9b08', 'https://open.spotify.com/playlist/37i9dQZF1DX0P7PzzKwEKl', 
                'https://open.spotify.com/playlist/37i9dQZF1DX9Ol4tZWPH6V', 'https://open.spotify.com/playlist/37i9dQZF1DX10zKzsJ2jva', 
                'https://open.spotify.com/playlist/37i9dQZF1DX4o1oenSJRJd']

In [87]:
artists_list = []
start_time = time.time()

for i in playlist_get:
    add_artist_name_from_playlist(i, artists_list)
    time.sleep(3)
    
print("--- %s seconds ---" % (time.time() - start_time))
print(len(artists_list))

# test_url = 'https://open.spotify.com/playlist/37i9dQZF1DX6Q49slBRXI2'
# test_url = test_url.rsplit('/', 1)[-1]
# playlist = sp.playlist(test_url)

--- 69.54240918159485 seconds ---
711


## Get by name

In [722]:
def get_artist_info(artist_name):
    try:
        top_result = sp.search(q = 'artist:' + artist_name, type = 'artist')['artists']['items'][0]
        info_dict = {'Name': top_result['name'], 'URI': top_result['uri'], 'Popularity': top_result['popularity'], 'Genres': top_result['genres'], 'Followers': top_result['followers']['total']}
        return(info_dict)
    except Exception as e:
        print(f'An error occured: {e} for the artist: {artist_name}')
        return None

In [814]:
artist_info = pd.read_csv('data/artist_info.csv')
print(f'artist_info len: {len(artist_info)}')

artist_info len: 1166


In [816]:
artist_info[artist_info.duplicated()]
# artist_info[artist_info.Name == "Drake"]

,Name,URI,Popularity,Genres,Followers


In [810]:
artists_list = [
    "Noriel", "Shania Twain", "Leon Bridges", "MC Ryan SP", "PinkPantheress",
    "Wham!", "Electric Light Orchestra", "Key Glock", "Hugo & Guilherme", "ODESZA",
    "Mon Laferte", "La Oreja de Van Gogh", "De La Ghetto", "Young Miko", "Samra",
    "La Adictiva Banda San José de Mesillas", "Dilsinho", "The xx", "Iggy Azalea", "Grupo Marca Registrada",
    "Seeb", "Sido", "Cavetown", "3 Doors Down", "Nick Jonas",
    "Jubin Nautiyal", "Julieta Venegas", "Ms. Lauryn Hill", "Ghost", "Dean Martin",
    "Of Monsters and Men", "keshi", "Gwen Stefani", "Joyner Lucas", "V",
    "Jon Pardi", "Bing Crosby", "Kontra K", "Latto", "M83",
    "Juan Luis Guerra 4.40", "JVKE", "BONES", "Shreya Ghoshal", "Ali Gatie",
    "The Goo Goo Dolls", "NIKI", "Mother Mother", "Marilyn Manson", "Tulus",
    "WALK THE MOON", "Nat King Cole", "Jon Bellion", "Rich The Kid", "Brytiago",
    "Sade", "Dido", "Mötley Crüe", "Guilherme & Benuto", "Alice In Chains",
    "d4vd", "Lofi Fruits Music", "Sufjan Stevens", "Canserbero", "a-ha",
    "Ella Henderson", "Sleeping At Last", "Stormzy", "Pedro Capó", "Juan Magán",
    "Cosculluela", "Chayanne", "Queens of the Stone Age", "Yuridia", "Mustard",
    "Ke Personajes", "Ghostemane", "The Clash", "Pineapple StormTv", "Powfu",
    "Gustavo Mioto", "Rascal Flatts", "CRO", "iann dior", "Ñengo Flow",
    "Young Dolph", "Intocable", "Timmy Trumpet", "Ella Mai", "Aleman",
    "Darshan Raval", "Santana", "Ezhel", "Kylie Minogue", "John Lennon",
    "Westlife", "Louis Armstrong", "Natalia Lafourcade", "KYLE", "Axwell / Ingrosso",
    "Two Feet", "Yung Gravy", "IU", "David Bisbal", "Louis Tomlinson",
    "Two Door Cinema Club", "Milo j", "MC Kevin o Chris", "Israel & Rodolffo", "Gazo",
    "Benjamin Blümchen", "Mari Fernandez", "Mambo Kingz", "Taio Cruz", "Colbie Caillat",
    "Bill Withers", "Casper Magico", "Beach House", "Lee Brice", "Cali Y El Dandee",
    "Annie Lennox", "Kevin Kaarl", "Roxette", "Timbaland", "ITZY",
    "Owl City", "AURORA", "Fergie", "Beret", "DJ Luian",
    "Los Enanitos Verdes", "DMX", "Falling In Reverse", "Miranda Lambert", "Astrid S",
    "Don Diablo", "Benson Boone", "Gigi D'Agostino", "The Pussycat Dolls", "Los Dos Carnales",
    "Nekfeu", "Sting", "Tee Grizzley", "BoyWithUke", "Stephen Sanchez",
    "Sin Bandera", "Matoma", "Badshah", "Pierce The Veil", "Hardwell",
    "Sean Kingston", "Old Dominion", "Booba", "Official HIGE DANdism", "KK",
    "Cody Johnson", "Boney M.", "Sachin-Jigar", "Deorro", "Los Bukis",
    "Mc Don Juan", "Alex Rose", "Loud Luxury", "LE SSERAFIM", "PLK",
    "The Kooks", "Incubus", "Vintage Culture", "Cyndi Lauper", "Diego & Victor Hugo",
    "PnB Rock", "Niska", "Selena", "Denzel Curry", "Conor Maynard",
    "NCT DREAM", "Dierks Bentley", "ANAVITÓRIA", "Oscar Maydon", "Jelly Roll",
    "Ferrugem", "Yuvan Shankar Raja", "The Who", "Calle 13", "Ben Howard",
    "Portugal. The Man", "Snow Patrol", "Wallows", "Lionel Richie", "Afrojack",
    "Hungria Hip Hop", "Kungs", "Yusuf / Cat Stevens", "Kevin Roldan", "Camila",
    "Tina Turner", "Quinn XCII", "A7S", "UZI", "Blessd",
    "Jaymes Young", "Ofenbach", "Filipe Ret", "Arizona Zervas", "Saweetie",
    "John Newman", "L7NNON", "Ice Spice", "Chelsea Cutler", "Dread Mar I",
    "Die drei !!!", "The Jackson 5", "NCT 127", "Karan Aujla", "Vampire Weekend",
    "Jessie Reyez", "Seether", "Jesús Adrián Romero", "Surfaces", "Jaden",
    "Ciara", "Jordan Davis", "Wale", "fun.", "Martin Jensen",
    "KALEO", "Lazza", "Lady A", "Internet Money", "Bronco",
    "Otis Redding", "Tracy Chapman", "Fred again..", "Hombres G", "Granular",
    "gnash", "Daya", "Willy William", "Thiaguinho", "RÜFÜS DU SOL",
    "Sid Sriram", "Pixies", "Sezen Aksu", "Beastie Boys", "Mary J. Blige",
    "Gloria Trevi", "DENNIS", "R. Kelly", "*NSYNC", "Regard",
    "WizKid", "Mrs. GREEN APPLE", "Blur", "Emilia", "Simple Plan",
    "Kollegah", "Thalia", "Summer Cem", "Olly Murs", "Tech N9ne",
    "Diljit Dosanjh", "LIT killah", "Ben&Ben", "ZZ Top", "Nina Simone",
    "Toby Keith", "Pepe Aguilar", "Paul Simon", "Gustavo Cerati", "Trevor Daniel",
    "Leona Lewis", "Sonu Nigam", "James TW", "Lenny Kravitz", "OMI",
    "deadmau5", "Dadju", "Carla Morrison", "The Score", "Oliver Heldens",
    "Bo Burnham", "Simone Sommerland", "Dynoro", "Maren Morris", "ATEEZ",
    "AJ Tracey", "Djadja & Dinaz", "Sofía Reyes", "Mike Perry", "JID",
    "Kina", "SCH", "Supertramp", "Laura Pausini", "MC Kevinho",
    "Chris Young", "Shubh", "MAGIC!", "Cristian Castro", "Capo Plaza",
    "Cher", "Blueface", "Freddie Dredd", "Karsten Glück", "Bullet For My Valentine",
    "Sara Bareilles", "Lauren Daigle", "Gabry Ponte", "John Denver", "MKTO",
    "Banda Los Recoditos", "Desiigner", "Tom Petty", "João Gomes", "Kelsea Ballerini",
    "Zé Felipe", "Cuco", "ONE OK ROCK", "RADWIMPS", "Sorriso Maroto",
    "Neha Kakkar", "$NOT", "Foreigner", "Andrea Bocelli", "Dennis Lloyd",
    "PEDRO SAMPAIO", "Kate Bush", "Darell", "VIZE", "Audioslave",
    "CKay"
]

print(len(artists_list))

start_time = time.time()
to_add_artist_info = []

for name in artists_list:
    if name not in artist_info['Name']:
        to_add_artist_info.append(get_artist_info(name))
    else:
        pass

print("--- %s seconds ---" % (time.time() - start_time))
print(f'Length of to_add: {len(to_add_artist_info)} dudes')

331
--- 37.51091027259827 seconds ---
Length of to_add: 331 dudes


In [728]:
ad_hoc_artists_list = ['George Michael', 'Wham!', 'The Jacksons', 'The Jackson 5']

for name in ad_hoc_artists_list:
    if name not in artist_info['Name']:
        to_add_artist_info.append(get_artist_info(name))
    else:
        pass

print(f'Length of to_add: {len(to_add_artist_info)} dudes')

--- 156.00960111618042 seconds ---
Length of to_add: 92 dudes


In [818]:
artist_info_add = pd.json_normalize(to_add_artist_info)
print(f'Old len: {len(artist_info)}')
artist_info_new = pd.concat([artist_info, artist_info_add], ignore_index = True)

for col in artist_info_new.columns:
    if artist_info_new[col].apply(lambda x: isinstance(x, list)).any():
        artist_info_new[col] = artist_info_new[col].apply(str)

artist_info_new = artist_info_new.drop_duplicates(subset = ['URI'])

print(f'New len: {len(artist_info_new)}')
artist_info_new.to_csv('data/artist_info.csv', index = False)

Old len: 1166
New len: 1166


## Get by URI

In [5]:
artist_df = pd.read_csv('data/artist_info.csv')
print(f'artist_df len: {len(artist_df)}')

artist_df len: 1166


In [173]:
album_df = pd.read_csv('data/album_info.csv')
album_df.drop_duplicates(inplace = True)
album_df.loc[:, 'artist_name'] = album_df.artists.apply(lambda x: [artist['name'] for artist in ast.literal_eval(x)])
album_df.loc[:, 'artist_uri'] = album_df.artists.apply(lambda x: [artist['uri'] for artist in ast.literal_eval(x)])
print(f'artist_df len: {len(album_df)}')
# album_df.head()

artist_df len: 13269


In [169]:
# Finding artists not in the file
alb_artists = set()

for artists in album_df['artist_uri']:
    for art in artists:
        alb_artists.add(art)

artists_in_df = set(artist_df['URI'])

artists_not_in = alb_artists.difference(artists_in_df)

check_artists = set()

for i in artists_not_in:
    if artist_df['URI'].eq(i).any() == True:
        pass
    else:
        check_artists.add(i)

print(len(check_artists))
print(len(artists_not_in))
print(check_artists == artists_not_in)

2354
2354
True


In [185]:
def get_artist_uri(artist_uri):
    try:
        response = sp.artist(artist_uri)
        info_dict = {'Name': response['name'], 
                     'URI': response['uri'], 
                     'Popularity': response['popularity'], 
                     'Genres': response['genres'], 
                     'Followers': response['followers']['total']}
        return(info_dict)
    except Exception as e:
        print(f'An error occured: {e} for the artist: {artist_uri}')
        return None

In [211]:
# Completed

start_time = time.time()
add_artists_from_albums = []

for art_uri in artists_not_in:
    add_artists_from_albums.append(get_artist_uri(art_uri))

print("--- %s seconds ---" % (time.time() - start_time))
print(f'Length of add_artists_from_albums: {len(add_artists_from_albums)} dudes')

--- 199.49376678466797 seconds ---
Length of add_artists_from_albums: 2354 dudes


In [221]:
artist_info_add = pd.json_normalize(add_artists_from_albums)
print(f'Old len: {len(artist_df)}')
artist_info_new = pd.concat([artist_df, artist_info_add], ignore_index = True)

for col in artist_info_new.columns:
    if artist_info_new[col].apply(lambda x: isinstance(x, list)).any():
        artist_info_new[col] = artist_info_new[col].apply(str)

artist_info_new = artist_info_new.drop_duplicates(subset = ['URI'])

print(f'New len: {len(artist_info_new)}')
artist_info_new.to_csv('data/artist_info.csv', index = False)

Old len: 1166
New len: 3519


In [179]:
test_response = sp.artist('spotify:artist:23aPUZaR8bESXN4UD3T2Sx')

In [193]:
test_func = get_artist_uri('spotify:artist:23aPUZaR8bESXN4UD3T2Sx')

In [195]:
test_func

{'Name': '香取慎吾',
 'URI': 'spotify:artist:23aPUZaR8bESXN4UD3T2Sx',
 'Popularity': 29,
 'Genres': [],
 'Followers': 25129}

In [181]:
test_response

{'external_urls': {'spotify': 'https://open.spotify.com/artist/23aPUZaR8bESXN4UD3T2Sx'},
 'followers': {'href': None, 'total': 25129},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/23aPUZaR8bESXN4UD3T2Sx',
 'id': '23aPUZaR8bESXN4UD3T2Sx',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebb08fef18649cc4c3b2396258',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174b08fef18649cc4c3b2396258',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178b08fef18649cc4c3b2396258',
   'width': 160}],
 'name': '香取慎吾',
 'popularity': 29,
 'type': 'artist',
 'uri': 'spotify:artist:23aPUZaR8bESXN4UD3T2Sx'}

# Albums df

In [15]:
def get_albums(artist_uri):
    results = sp.artist_albums(artist_uri)
    all_albums = []
    try:
        for album in results['items']:
            all_albums.append({'name': album['name'],
                               'id': album['id'],
                               'uri': album['uri'],
                               'album_group': album['album_group'],
                               'album_type': album['album_type'],
                               'type': album['type'],
                               'artists': album['artists'],
                               'release_date': album['release_date'],
                               'release_date_precision': album['release_date_precision'],
                               'total_tracks': album['total_tracks'],
                               'external_urls': album['external_urls'],
                               'artist_name': [artist['name'] for artist in album['artists']],
                               'artist_uri': [artist['uri'] for artist in album['artists']]
                              })
        return(all_albums)
    except Exception as e:
        print(f"An error occured: {e} for the album: {album['name']}")
        return None

In [ ]:
#############
# Completed #
#############

# Initial run
#  711 artists -> 13,841 albums: 184.90106439590454 seconds

# Since then, the artist_df file has been revamped so need to find the difference in artists

In [11]:
artist_df = pd.read_csv('data/artist_info.csv')
album_df = pd.read_csv('data/album_info.csv')
# album_df.loc[:, 'artist_name'] = album_df.artists.apply(lambda x: [artist['name'] for artist in ast.literal_eval(x)])
# album_df.loc[:, 'artist_uri'] = album_df.artists.apply(lambda x: [artist['uri'] for artist in ast.literal_eval(x)])
print(len(album_df))

35232


In [ ]:
# Completed

# artist_df[0:2000]['URI'] -> 573.5985157489777 seconds, 37557 albums

In [13]:
start_time = time.time()
album_list = []

for art_uri in artist_df[2000:4000]['URI']:
    response = get_albums(art_uri)
    album_list.extend(response)

print("--- %s seconds ---" % (time.time() - start_time))
print(f'Length of add_artists_from_albums: {len(album_list)} albums')

NameError: name 'get_albums' is not defined

In [ ]:
## Initial run
# album_list = []
# artist_df = artist_df.dropna()

# start_time = time.time()

# for uri in artist_df['URI']:
#     album_list += get_albums(uri)
    
# print("--- %s seconds ---" % (time.time() - start_time))
# print(f'album_list len: {len(album_list)}')

# # 711 artists -> 13,841 albums: 184.90106439590454 seconds

In [ ]:
add_album_df = pd.json_normalize(album_list)

print(f'Old len: {len(album_df)}')
album_df_new = pd.concat([album_df, add_album_df], ignore_index = True)
album_df_new['artist_name'] = album_df_new['artist_name'].apply(str)
album_df_new['artist_uri'] = album_df_new['artist_uri'].apply(str)
album_df_new = album_df_new.drop_duplicates(subset = 'uri')
album_df_new['artist_name'] = album_df_new['artist_name'].apply(ast.literal_eval)
album_df_new['artist_uri'] = album_df_new['artist_uri'].apply(ast.literal_eval)
print(f'New len: {len(album_df_new)}')
album_df_new.to_csv('data/album_info.csv', index = False)

In [331]:
# album_df_new[album_df_new.uri.duplicated()]
add_album_df = pd.json_normalize(album_list)
add_album_df

,name,id,uri,album_group,album_type,type,artists,release_date,release_date_precision,total_tracks,artist_name,artist_uri,external_urls.spotify
0,Proof,6al2VdKbb6FIz9d7lU7WRB,spotify:album:6al2VdKbb6FIz9d7lU7WRB,album,album,album,[{'external_urls': {'spotify': 'https://open.s...,2022-06-10,day,35,[BTS],[spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX],https://open.spotify.com/album/6al2VdKbb6FIz9d...
1,BE,6nYfHQnvkvOTNHnOhDT3sr,spotify:album:6nYfHQnvkvOTNHnOhDT3sr,album,album,album,[{'external_urls': {'spotify': 'https://open.s...,2020-11-20,day,8,[BTS],[spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX],https://open.spotify.com/album/6nYfHQnvkvOTNHn...
2,MAP OF THE SOUL : 7 ~ THE JOURNEY ~,1nScVw87kRJiT2bg2Kswhp,spotify:album:1nScVw87kRJiT2bg2Kswhp,album,album,album,[{'external_urls': {'spotify': 'https://open.s...,2020-07-14,day,13,[BTS],[spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX],https://open.spotify.com/album/1nScVw87kRJiT2b...
3,MAP OF THE SOUL : 7,5W1XY5ucNATjTULERvXx9j,spotify:album:5W1XY5ucNATjTULERvXx9j,album,album,album,[{'external_urls': {'spotify': 'https://open.s...,2020-02-21,day,20,[BTS],[spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX],https://open.spotify.com/album/5W1XY5ucNATjTUL...
4,MAP OF THE SOUL : PERSONA,2KqlAl1Kl5fZvbFgJ0qFB6,spotify:album:2KqlAl1Kl5fZvbFgJ0qFB6,album,album,album,[{'external_urls': {'spotify': 'https://open.s...,2019-04-12,day,7,[BTS],[spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX],https://open.spotify.com/album/2KqlAl1Kl5fZvbF...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19574,Blah Blah Blah,0KaoPehrtwhjQfae6XMd7B,spotify:album:0KaoPehrtwhjQfae6XMd7B,single,single,album,[{'external_urls': {'spotify': 'https://open.s...,2022-09-21,day,1,[ITZY],[spotify:artist:2KC9Qb60EaY0kW4eH68vr3],https://open.spotify.com/album/0KaoPehrtwhjQfa...
19575,Break My Heart Myself (feat. YEJI & RYUJIN of ...,6nHECY2OnWXVmd4QjGhJEm,spotify:album:6nHECY2OnWXVmd4QjGhJEm,single,single,album,[{'external_urls': {'spotify': 'https://open.s...,2022-07-29,day,1,"[Bebe Rexha, ITZY]","[spotify:artist:64M6ah0SkkRsnPGtGiRAbb, spotif...",https://open.spotify.com/album/6nHECY2OnWXVmd4...
19576,Voltage,6BhKPFeQDaBldGMl1vd28h,spotify:album:6BhKPFeQDaBldGMl1vd28h,single,single,album,[{'external_urls': {'spotify': 'https://open.s...,2022-04-06,day,4,[ITZY],[spotify:artist:2KC9Qb60EaY0kW4eH68vr3],https://open.spotify.com/album/6BhKPFeQDaBldGM...
19577,Voltage,3MXVqfk9VG3B757nLlow0D,spotify:album:3MXVqfk9VG3B757nLlow0D,single,single,album,[{'external_urls': {'spotify': 'https://open.s...,2022-03-23,day,1,[ITZY],[spotify:artist:2KC9Qb60EaY0kW4eH68vr3],https://open.spotify.com/album/3MXVqfk9VG3B757...


# Extracting individual tracks from albums

In [13]:
def get_tracks_from_album(curr_album_uri, curr_album_name):
    try:
        response = sp.album_tracks(curr_album_uri)['items']
        tracks_list = []
        for song in response:
            tracks_list.append({'name': song['name'],
                                'album_name': curr_album_name,
                                'album_uri': curr_album_uri,
                                'artists': song['artists'],
                                'uri': song['uri'],
                                'id': song['id'],
                                'disc_num': song['disc_number'],
                                'song_num': song['track_number'],
                                'type': song['type'], # probably not necessary
                                'duration_ms': song['duration_ms'],
                                'explicit': song['explicit'],
                                'is_local': song['is_local'],
                                'external_urls': song['external_urls'],
                                'href': song['href']})
        return(tracks_list)
    except Exception as e:
        print(f"An error occured: {e}")
        return(None)

In [37]:
# Load in album_df
album_df = pd.read_csv('data/album_info.csv')
album_df.rename(columns = {'uri': 'album_uri'}, inplace = True)
album_tracks = pd.read_csv('data/album_tracks.csv')
print(f'album_df len: {len(album_df)}')
print(f'song_df len: {len(album_tracks)}')

missing_album_uris = album_df[~album_df['album_uri'].isin(album_tracks['album_uri'])]
print(f'missing len: {len(missing_album_uris)}')

album_df len: 35232
song_df len: 181989


In [ ]:
"""
########
# Note #
########

Due to low Spotify user count (which at beginning of development = 0), they set an API call limit on you
This allows for about 1000 - 2500 albums to have their song data to be called until they put a temporary freeze on you for about 1 day (~24 hours)

"""

In [ ]:
"""
 -----------
| Completed |
 -----------

5/14/2024 5:42 PM
album_df[0:2000].iterrows() -> 311.8622350692749 seconds
21334 songs

5/15/2024 4:46 PM
album_df[2000:4000].iterrows(): -> 425.7372999191284 seconds
15457 songs

5/23/2024 ~7:30 PM
album_df[4000:6000].iterrows(): -> 271.8079960346222 seconds
11974 songs

5/24/2024 8:14 PM
album_df[6000:8000].iterrows(): -> 285.08418321609497 seconds
19734 songs

5/26/2024 6:45 AM
album_df[8000:10000].iterrows(): -> 263.82689666748047 seconds
21391 songs

5/26/2024 6:24 PM
album_df[10000:12000].iterrows(): -> 286.41857385635376 seconds
22109 songs
__________________________________________________________________ ^ pre album rework

Using missing_album_uris
 -> will always use missing_album_uris[0:2000] because missing_album_uris will always change as the dependent dataframes update
 
Run 1 -> 303.7286882400513 seconds, 17548 songs
Run 2 -> 290.0477089881897 seconds, 21237 songs
Run 3 -> 290.2157793045044 seconds, 20242 songs
Run 4 -> 286.6179428100586 seconds, 17763 songs

"""

In [33]:
start_time = time.time()
tracks_to_add = []

for idx, row in missing_album_uris[0:2000].iterrows():
    curr_album_uri = row.album_uri
    curr_album_name = row['name']
    tracks = get_tracks_from_album(curr_album_uri, curr_album_name)
    tracks_to_add.extend(tracks)

print("--- %s seconds ---" % (time.time() - start_time))
print(f'Length of tracks_to_add: {len(tracks_to_add)} songs')

--- 286.6179428100586 seconds ---
Length of tracks_to_add: 17763 songs


In [35]:
# For the first time #
# album_tracks = pd.json_normalize(tracks_to_add)
# album_tracks.to_csv('data/album_tracks.csv', index = False)

# For additions #
add_tracks_df = pd.json_normalize(tracks_to_add)
print(f'Curr df len: {len(album_tracks)}')
album_tracks_new = pd.concat([album_tracks, add_tracks_df], ignore_index = True)

print(f'\nPost-Concat:\n Pre-clean: {len(album_tracks_new)}')

# cannot simply do album_tracks_new.drop_duplicates
#  produced TypeError: unhashable type: 'list'
#  most likely from the 'artists' column
#  convert list to string, then .drop_duplicates()

for col in album_tracks_new.columns:
    if album_tracks_new[col].apply(lambda x: isinstance(x, list)).any():
        album_tracks_new[col] = album_tracks_new[col].apply(str)

album_tracks_new = album_tracks_new.drop_duplicates()        

print(f' Post-clean: {len(album_tracks_new)}')
album_tracks_new.to_csv('data/album_tracks.csv', index = False)

Curr df len: 164226

Post-Concat:
 Pre-clean: 164226
 Post-clean: 181989


# Audio features

In [ ]:
"""
_________________________________________________________________

start_time = time.time()
audio_feats_to_add = []

range_start = 0
while range_start < 500:
    audio_feats_to_add.extend(sp.audio_features(album_tracks.uri[range_start:range_start + 100]))
    range_start += 100
    
print("--- %s seconds ---" % (time.time() - start_time))

_________________________________________________________________

#############
# Completed # -> pre-data_rework
#############

# album_tracks.uri index -> time it took

5/27/2024
[0:20000] -> 62.38555073738098 seconds
[20000:40000] -> 57.44641590118408 seconds

_________________________________________________________________

#############
# Completed # -> post-date_rework
#############

Using difference between tracks present in album_tracks and tracks not in feat_df

Doing it in chunks of 20k songs

Run 1 -> 63.74984288215637 seconds
Run 2 -> 63.01819944381714 seconds
Run 3 -> ~63 seconds
Run 4 -> 63.01237893104553 seconds
Run 5 -> 62.35223627090454 seconds
Run 6 -> 58.528666973114014 seconds
Run 7 -> 56.75151991844177 seconds

"""

In [61]:
album_tracks = pd.read_csv('data/album_tracks.csv')
feat_df = pd.read_csv('data/feat_df.csv')
print(f'Length of album_tracks: {len(album_tracks)}')
print(f'Length of feat_df: {len(feat_df)}')

Length of album_tracks: 181989
Length of feat_df: 158479


In [67]:
missing_track_uris = album_tracks[~album_tracks['uri'].isin(feat_df['uri'])]
print(len(missing_track_uris))

23510


In [69]:
start_time = time.time()
audio_feats_to_add = []

# sp.audio_features can only intake 100 ids at a time
range_start = 0
range_end = 20000
while range_start < range_end:
    audio_feats_to_add.extend(sp.audio_features(missing_track_uris.uri[range_start:range_start + 100]))
    range_start += 100
    
print("--- %s seconds ---" % (time.time() - start_time))
# print(f'Length of to-add song_feats: {len(audio_feats_to_add)}')

--- 56.75151991844177 seconds ---


In [71]:
# # # First run
# feats_add = pd.json_normalize(audio_feats_to_add)
# feat_df_new = pd.merge(feats_add, album_tracks[['uri', 'name', 'album_name', 'album_uri', 'artists']], on='uri', how='left')
# print(len(feat_df_new))
# feat_df_new.drop_duplicates(inplace = True) # clears duplicates and songs without analysis (returned a JSON full of null values)
# feat_df_new.dropna(inplace = True)
# print(len(feat_df_new))
# feat_df_new.to_csv('data/feat_df.csv', index = False)

# # Additions
feat_add = pd.json_normalize(audio_feats_to_add)
print(f'Curr df len: {len(feat_df)}')
feat_df_new = pd.concat([feat_df, feat_add], ignore_index = True)

# adding in artist, album data
feat_df_new = feat_df_new.merge(album_tracks[['uri', 'name', 'album_name', 'album_uri', 'artists']], on='uri', how='left', suffixes = ('', '_new'))
feat_df_new.drop(columns = [
    'name', 'album_name', 'album_uri', 'artists', 'id', 'type', 'track_href', 'analysis_url'
], inplace = True)
feat_df_new.rename(columns = {
    'name_new': 'name',
    'album_name_new': 'album_name',
    'album_uri_new': 'album_uri',
    'artists_new': 'artists'
}, inplace = True)

# Drop duplicates
print('\nPost concat:')
print(f'-> Pre-clean: {len(feat_df_new)}')
feat_df_new.drop_duplicates(inplace = True) # clears duplicates and songs without analysis (returned a JSON full of null values)
feat_df_new.dropna(inplace = True)
print(f'-> Post-clean: {len(feat_df_new)}')

# Save file
feat_df_new.to_csv('data/feat_df.csv', index = False)

Curr df len: 158479

Post concat:
-> Pre-clean: 178485
-> Post-clean: 178119


## Debugging

### Missing, null, duplicates

In [660]:
feat_df_new[feat_df_new.isnull().any(axis = 1)]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,name,alubm_name,album_uri,artists


In [670]:
feat_df_new.iloc[-1]

danceability                                                    0.513
energy                                                           0.25
key                                                               5.0
loudness                                                       -9.376
mode                                                              0.0
speechiness                                                    0.0367
acousticness                                                    0.913
instrumentalness                                                  0.0
liveness                                                        0.117
valence                                                         0.351
tempo                                                          74.103
type                                                   audio_features
id                                             0pqTqDgxR1fcKZuBxgJ2QL
uri                              spotify:track:0pqTqDgxR1fcKZuBxgJ2QL
track_href          

In [658]:
album_tracks[20000:40000].iloc[-1]

name                                                                 Sorry
album_name                                hopeless fountain kingdom (Plus)
album_uri                             spotify:album:5gnbGc6VR6SeHCbx0DV67s
artists                  [{'external_urls': {'spotify': 'https://open.s...
uri                                   spotify:track:0pqTqDgxR1fcKZuBxgJ2QL
id                                                  0pqTqDgxR1fcKZuBxgJ2QL
disc_num                                                                 1
song_num                                                                 6
type                                                                 track
duration_ms                                                         220540
explicit                                                             False
is_local                                                             False
href                     https://api.spotify.com/v1/tracks/0pqTqDgxR1fc...
external_urls.spotify    

### Data structure

# Audio analyses

In [ ]:
"""
the audio analyses obtained with sp.audio_analysis(uri) contains "more technical" information than audio features
however, this data is only useful if you have a high and technical understanding of music and the echoprint system

most of the legible and "useful" data are already present in audio features
"""

# Misc